In [1]:
pip install -q findspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
findspark.init('/home/bigdata/Documents/spark-3.0.0')

In [3]:
from pyspark import SparkConf, SparkContext, Row
from pyspark.streaming import StreamingContext
import os
from pyspark import*
from pyspark.sql.functions import col, when
import pandas as pd

In [4]:
config = (SparkConf().setAppName("UNSW").set("spark.executor.memory", "2g"))

sc = SparkContext(conf=config)
ssc = StreamingContext(sc, 2)
ssc.checkpoint("checkpoint")

In [5]:
# Load data to create a DStream data

unsw_list = list()
# read the entire folder and add them into the Queue
entries = os.listdir('UNSW_data')
for entry in entries:
    print(entry)
    path="UNSW_data/" + entry
    unsw_list += [sc.textFile(path, 5)]

UNSW-NB15-8.csv
UNSW-NB15-43.csv
UNSW-NB15-39.csv
UNSW-NB15-3.csv
UNSW-NB15-1.csv
UNSW-NB15-14.csv
UNSW-NB15-17.csv
UNSW-NB15-25.csv
UNSW-NB15-27.csv
UNSW-NB15-46.csv
UNSW-NB15-4.csv
UNSW-NB15-7.csv
UNSW-NB15-29.csv
UNSW-NB15-9.csv
UNSW-NB15-38.csv
UNSW-NB15-6.csv
UNSW-NB15-49.csv
UNSW-NB15-2.csv
UNSW-NB15-5.csv
UNSW-NB15-30.csv
.~lock.UNSW-NB15-1.csv#
UNSW-NB15-10.csv
UNSW-NB15-18.csv
UNSW-NB15-28.csv
UNSW-NB15-35.csv
UNSW-NB15-33.csv
UNSW-NB15-22.csv
UNSW-NB15-19.csv


In [6]:
unsw_list[2].take(3)

['"59.166.0.8","48111","149.171.126.7","5190","tcp","FIN","1.6395999E-2","1036","2366","31","29","4","4","-","463527.72","1071968.9","12","14","255","255","2472443027","327436606","86","169","0","0","127.15737","3.16449","1421947909","1421947909","1.453818","1.209846","7.8599999E-4","6.5900001E-4","1.2700001E-4","0","0","0","0","0","7","9","3","1","1","1","1","","0"',
 '"59.166.0.7","14767","149.171.126.5","80","tcp","FIN","1.1078809","1684","10168","31","29","3","4","http","11293.632","69350.414","14","18","255","255","1514550030","3665297448","120","565","1","3924","9043.1875","7918.4985","1421947908","1421947909","85.194153","65.13736","6.9000002E-4","0.000543","0.000147","0","0","1","0","0","2","1","2","1","1","1","1","","0"',
 '"59.166.0.1","18659","149.171.126.7","14562","tcp","FIN","0.012538","2542","23122","31","29","7","14","-","1581751.5","1.4402297E+7","40","42","255","255","2716648099","572382181","64","551","0","0","18.4088","17.361631","1421947910","1421947910","0.312846"

In [7]:
# Make the Queue Stream of data 
rdd_stream = ssc.queueStream(unsw_list)

In [8]:
def AddNormalLabel(line1): 
    line =line1.replace('"','').split(",")
    if line[47]== '""':
        line[47]= 'Normal'
    elif line[47] == 'Backdoor':
        line[47] ='Backdoors'
   
    return line

In [9]:
words = rdd_stream.map(AddNormalLabel)

wordfilt= words.filter(lambda word: word[47])

red = wordfilt.map(lambda word: word[47])
count= red.countByValue()
count.pprint()
#counts = count.reduceByKey(lambda x, y: x+y)
#counts.pprint()

In [10]:
ssc.start()

-------------------------------------------
Time: 2020-05-14 19:29:08
-------------------------------------------
('Fuzzers ', 325)
('Backdoors', 12)
('DoS', 107)
('Reconnaissance ', 254)
('Shellcode ', 25)
('Worms', 3)
('Generic', 13828)
('Exploits', 537)
('Analysis', 30)

-------------------------------------------
Time: 2020-05-14 19:29:10
-------------------------------------------
('DoS', 584)
('Fuzzers ', 404)
('Backdoors', 10)
('Reconnaissance ', 298)
('Worms', 3)
('Shellcode ', 26)
('Generic', 5621)
('Exploits', 1441)
('Analysis', 85)

-------------------------------------------
Time: 2020-05-14 19:29:12
-------------------------------------------

-------------------------------------------
Time: 2020-05-14 19:29:14
-------------------------------------------
('DoS', 602)
('Fuzzers ', 607)
('Backdoors', 74)
('Reconnaissance ', 406)
('Shellcode ', 46)
('Worms', 4)
('Exploits', 1299)
('Analysis', 74)
('Generic', 8032)

-------------------------------------------
Time: 2020-05-14

In [11]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2020-05-14 19:30:18
-------------------------------------------

-------------------------------------------
Time: 2020-05-14 19:30:20
-------------------------------------------

-------------------------------------------
Time: 2020-05-14 19:30:22
-------------------------------------------

